In [2]:
import os
import requests
import pandas as pd
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from git import Repo

In [18]:
CHECKPOINT = "neuralsentry/starencoder-finetuned-class"

model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

In [34]:
Repo.clone_from("https://github.com/redis/redis", "test/repos/redis")

<git.repo.base.Repo 'd:\\Projects\\neural-sentry\\test\\repos\\redis\\.git'>

In [39]:
repo = Repo("test/repos/redis")

In [40]:
TARGET_BUG_COMMITS = 50
buggy_commits = []

In [43]:
for commit in repo.iter_commits():
    commit_msg = commit.message
    outputs = model(
        **tokenizer(commit_msg, return_tensors="pt", truncation=True, max_length=512)
    )
    logits = outputs.logits
    pred = logits.argmax().item()
    if pred == 1:
        buggy_commits.append(commit)
        print(f"[+] Found {len(buggy_commits)} buggy commits")
    else:
        print(
            f'[-] Found "{commit_msg[:50] + "..." if len(commit_msg) > 50 else commit_msg} non-buggy commit"'
        )

    if len(buggy_commits) == TARGET_BUG_COMMITS:
        break

[+] Found 10 buggy commits
[-] Found "Fix typos in comments (#12338)

 non-buggy commit"
[+] Found 11 buggy commits
[+] Found 12 buggy commits
[+] Found 13 buggy commits
[-] Found "Align RM_ReplyWithErrorFormat with RM_ReplyWithErr... non-buggy commit"
[+] Found 14 buggy commits
[+] Found 15 buggy commits
[+] Found 16 buggy commits
[+] Found 17 buggy commits
[-] Found "Fix ZRANK/ZREVRANK reply_schema description (#1233... non-buggy commit"
[-] Found "Doc indentation fix for the --functions-rdb option... non-buggy commit"
[+] Found 18 buggy commits
[-] Found "Add ability for modules to know which client is be... non-buggy commit"
[+] Found 19 buggy commits
[-] Found "Optimize PSUBSCRIBE and PUNSUBSCRIBE from O(N*M) t... non-buggy commit"
[+] Found 20 buggy commits
[-] Found "Cluster human readable nodename feature  (#9564)

... non-buggy commit"
[+] Found 21 buggy commits
[+] Found 22 buggy commits
[-] Found "Removing duplicated tests (#12318)

In 4ba47d2d2 t... non-buggy commit"
[+] Fo

In [35]:
commit_msgs = [commit.message for commit in buggy_commits]
df = pd.DataFrame(commit_msgs, columns=["commit_msg"])

In [37]:
df.head(10)

,commit_msg
0,Modules: Unblock from within a timer coverage ...
1,Print strerror when bio initialization fails (...
2,Improve moduleBlockClient timeout overflow han...
3,Make nodename test more consistent (#12330)\n\...
4,Fix broken protocol when PUBLISH emits local p...
5,use embedded string object and more efficient ...
6,zrangeGenericCommand add check for negative of...
7,adding geo command edge cases tests (#12274)\n...
8,Sanitizer reported memory leak for '--invalid'...
9,Fix cluster human_nodename Getter data loss in...
